In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline

In [2]:
df = pd.read_csv("Crop_recommendation.csv")

# Train Test Split

In [6]:
from sklearn.model_selection import train_test_split

In [8]:
x_train, x_test, y_train, y_test = train_test_split(
    df.drop(columns='label'),
    df['label'],
    test_size=0.2
)

# column Transformers

In [11]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score

In [13]:
trf1 = ColumnTransformer(transformers=[
    ('standarisation', StandardScaler(with_mean=False), [0,6]),
],
    remainder='passthrough'
)

In [15]:
trf2 = ColumnTransformer(transformers=[
    ('ohe_label', OneHotEncoder(handle_unknown='ignore'), [-1])
], remainder='passthrough')

# importing models

In [20]:
from sklearn.neighbors import KNeighborsClassifier  # k-Nearest Neighbors (KNN)
from sklearn.svm import SVC  # Support Vector Machine (SVM)
from sklearn.tree import DecisionTreeClassifier  # Decision Tree Classifier
from sklearn.linear_model import LogisticRegression  # Logistic Regression
from sklearn.naive_bayes import GaussianNB  # Naive Bayes
from sklearn.ensemble import GradientBoostingClassifier  # Gradient Boosting Machine (GBM)
from sklearn.ensemble import RandomForestClassifier  # Random Forest Classifier


In [ ]:
models_dict = {
    
}